# Day 1

## Imports and data loading

In [2]:
from typing import List, Tuple, Dict

from utils import get_input, load_data

day = 14


In [3]:
get_input(day)


Data saved


In [3]:
data = load_data(day, list_type="line", number=False)
test_data = [
    "NNCB",
    "",
    "CH -> B",
    "HH -> N",
    "CB -> H",
    "NH -> C",
    "HB -> C",
    "HC -> B",
    "HN -> C",
    "NN -> C",
    "BH -> H",
    "NC -> B",
    "NB -> B",
    "BN -> B",
    "BB -> N",
    "BC -> B",
    "CC -> N",
    "CN -> C",
]
test_answer_1 = 1588
test_answer_2 = 2188189693529


## Part one

In [6]:
from collections import Counter


def organise_data(data: List[str]) -> Tuple[str, Dict[str, str]]:
    """Split data into starting string and dictionary of expansion rules."""
    start_string = data[0]
    rules = {}
    # Structure is always one line of input, then a blank line, then rules lines
    for d in data[2:]:
        divided = d.split(" -> ")
        rules[divided[0]] = divided[1]

    return start_string, rules


def step(start_string: str, rules: Dict[str, str]):
    """Insert rules value between each pair of letters that matches a rules key."""
    result = []
    for i, letter in enumerate(start_string[:-1]):
        # For each letter, look up that letter plus the next letter
        insertion = rules.get(letter + start_string[i + 1], "")
        result.append(letter + insertion)
    # Add the last letter at the end
    result.append(start_string[-1])
    return "".join(result)


def calculate_answer(polymer_string: str) -> int:
    """Subtract least common letter from most common letter"""
    letters = Counter(polymer_string)
    counts = letters.most_common()
    return counts[0][1] - counts[-1][1]


def run_part_one(data: List[str], repeats: int) -> int:
    """Run the whole process on a given list of input data."""
    polymer_chain, rules = organise_data(data)
    for r in range(repeats):
        polymer_chain = step(polymer_chain, rules)
    return calculate_answer(polymer_chain)


In [7]:
assert run_part_one(test_data, 10) == test_answer_1
run_part_one(data, 10)


3259

## Part two

In [24]:
# Oh no, it's an efficiency question...
# But order doesn't matter, so let's just dict it

# def efficient_step(letters: Dict[str, int], rules: Dict[str, str]) -> Dict[str, int]:
def make_dict(start_string: str) -> Dict[str, int]:
    """Turn the starting string into a dictionary of letter pairs and counts."""
    letters = {}
    for i, letter in enumerate(start_string[:-1]):
        letters[(letter, start_string[i + 1])] = (
            letters.get((letter, start_string[i + 1]), 0) + 1
        )
    return letters


def efficient_step(
    letters: Dict[Tuple[int, int], int], rules: Dict[str, str]
) -> Dict[Tuple[int, int], int]:
    """Run a dictionary of letter pair counts through the rules dictionary."""
    new_letters = {}
    for letter_pair, count in letters.items():
        pair_as_string = "".join(list(letter_pair))
        new_letters[(letter_pair[0], rules[pair_as_string])] = (
            new_letters.get((letter_pair[0], rules[pair_as_string]), 0) + count
        )
        new_letters[(rules[pair_as_string], letter_pair[1])] = (
            new_letters.get((rules[pair_as_string], letter_pair[1]), 0) + count
        )
    return new_letters


def calculate_answer_from_dict(
    letter_counts: Dict[Tuple[str, str], int], last_letter: str
) -> int:
    """Subtract least common letter from most common letter, using a letter-pair dict."""
    counts = {}
    for letter_pair, count in letter_counts.items():
        counts[letter_pair[0]] = counts.get(letter_pair[0], 0) + count
    counts[last_letter] = counts.get(last_letter, 0) + 1
    ordered = sorted(counts.items(), key=lambda x: x[1])
    return ordered[-1][1] - ordered[0][1]


def run_part_two(data: List[str], repeats: int) -> int:
    """Run the whole process on a given list of input data."""
    polymer_chain, rules = organise_data(data)
    letters = make_dict(polymer_chain)
    # Last letter won't change
    last_letter = polymer_chain[-1]
    for r in range(repeats):
        letters = efficient_step(letters, rules)
    return calculate_answer_from_dict(letters, last_letter)


In [27]:
assert run_part_two(test_data, 10) == test_answer_1
assert run_part_two(test_data, 40) == test_answer_2
run_part_two(data, 40)


3459174981021